In [ ]:
#
# Universidad EAFIT 
# 2025-2
# SI7016 - NLP - Lecture 07
#

# 1. Construir un sistema RAG que permita buscar información en una base de datos real de noticias, recuperando los artículos más relevantes y generando respuestas usando GPT-4.

* Usa LangChain y FAISS para la recuperación.
* Procesa un dataset real de noticias.
* Genera respuestas basadas en noticias relevantes.
* Evalúa la precisión con métricas como Recall@K y ROUGE.

In [ ]:
# 1. Instalación de dependencias
%pip install chromadb langchain langchain_community openai pandas tiktoken

# 2 - Descarga y Preprocesamiento del Dataset

Usaremos el dataset "News Category Dataset" de Kaggle, que contiene noticias reales.

Descarga el dataset desde Kaggle:

https://www.kaggle.com/datasets/rmisra/news-category-dataset?resource=download

descarga el archivo: News_Category_Dataset_v3.json.zip

# Cargar y Preprocesar el Dataset

El dataset de noticias viene en formato JSON, por lo que lo cargaremos y seleccionaremos las columnas necesarias.

In [ ]:
import pandas as pd
import json

# Cargar el dataset de noticias
file_path = "News_Category_Dataset_v3.json"  # Asegúrate de cambiar esto si el nombre del archivo es diferente

# Leer el archivo JSON línea por línea
data = []
with open(file_path, "r") as file:
    for line in file:
        data.append(json.loads(line))

# Convertir a un DataFrame de pandas
df = pd.DataFrame(data)

# Seleccionar columnas relevantes
df = df[['headline', 'short_description', 'category']]
df = df.dropna()  # Eliminar filas con valores nulos

# Crear una columna combinada con el título y la descripción para indexación
df["content"] = df["headline"] + ". " + df["short_description"]

print(df.head())

# volverlo más pequeño para poderlo procesar en tiempos razonables de clase y en una laptop
df = df[:10000]


In [ ]:
print(df.describe)

# 3. Crear la Base de Datos Vectorial en ChromaDB

Usamos ChromaDB para almacenar y recuperar embeddings de noticias.

In [ ]:
import chromadb
from langchain.embeddings import OpenAIEmbeddings

# Inicializar ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_news_db")

# Crear una colección en ChromaDB
collection = chroma_client.get_or_create_collection(name="news_articles")

# Generar embeddings para cada noticia y almacenarlas en la base de datos
embedding_model = OpenAIEmbeddings()
i=0
# en una laptop 16 GB RAM Intel i5, 35seg por cada 100 registros, 
# cargar 10.000 registros, toma aprox = 1 hora
for idx, row in df.iterrows():
    if (i%100==0):
        print(i)
    i=i+1
    collection.add(
        ids=[str(idx)],
        documents=[row["content"]],
        metadatas=[{"category": row["category"], "headline": row["headline"]}]
    )

print("Base de datos de noticias creada con éxito.")


Inicializamos ChromaDB en modo persistente.
Generamos embeddings con OpenAI y almacenamos los textos en ChromaDB.
Guardamos metadatos como categoría y título para mejorar la interpretación de los resultados.

# 4. Búsqueda Semántica con Recuperación de Noticias

Ahora creamos una función para buscar las noticias más relevantes usando búsqueda vectorial en ChromaDB.

In [ ]:
def search_news(query, top_k=3):
    results = collection.query(
        query_texts=[query],
        n_results=top_k
    )
    return results["documents"][0], results["metadatas"][0]

# Prueba de búsqueda
query = "News about Trump, white house"
retrieved_docs, metadata = search_news(query)

print("\nNoticias recuperadas:")
for i in range(len(retrieved_docs)):
    print(f"\n{metadata[i]['headline']}")
    print(f"\n{retrieved_docs[i]}")


search_news() consulta ChromaDB para recuperar los artículos más relevantes.
Ejemplo de consulta: Buscamos noticias sobre "elecciones en EE.UU." y mostramos los títulos y descripciones.

# 5. Generación de Respuestas con OpenAI GPT-4

Integramos ahora la recuperación con GPT-4 para responder preguntas basadas en las noticias.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

# Configurar el modelo de lenguaje
llm = ChatOpenAI(model_name="gpt-4")

# Crear la función de RAG
def rag_query(user_query, top_k=3):
    retrieved_docs, metadata = search_news(user_query, top_k)
    
    # Formatear el contexto para el modelo GPT-4
    context = "\n".join(retrieved_docs)
    
    # Crear el prompt para el modelo
    prompt = f"""
    Basado en las siguientes noticias, responde la siguiente pregunta de manera clara y concisa:

    Noticias:
    {context}

    Pregunta: {user_query}
    """

    # Generar la respuesta
    response = llm.predict(prompt)
    
    return response

# Prueba de generación
user_query = "¿Cuáles son las últimas noticias sobre el cambio climático?"
response = rag_query(user_query)

print("\nRespuesta generada por RAG:")
print(response)


Usamos GPT-4 para generar respuestas basadas en las noticias recuperadas.
rag_query() recupera noticias, formatea un prompt y consulta a GPT-4.
Prueba con consulta real: "¿Cuáles son las últimas noticias sobre el cambio climático?"

# 6. Evaluación del Sistema RAG

Evaluamos la recuperación y generación con Recall@K y ROUGE.

## 6.1 Evaluación de la Recuperación con Recall@K

In [ ]:
def evaluate_recall(queries, ground_truths, k=3):
    retrieved_texts = [search_news(q, k)[0] for q in queries]
    recall_k = sum(any(gt in retrieved for retrieved in retrieved_texts) for gt in ground_truths) / len(queries)
    
    print(f"\nRecall {k}: {recall_k:.2f}")

# Definir consultas y respuestas esperadas
test_queries = ["Noticias sobre economía", "Últimos eventos deportivos"]
expected_answers = ["Economía en crecimiento", "Partido de fútbol"]

evaluate_recall(test_queries, expected_answers, k=3)


Recall@K mide la capacidad del sistema de recuperar información correcta.
Si el recall es alto (cercano a 1), la recuperación es precisa.

## 6.2 Evaluación de la Generación con ROUGE

In [ ]:
from rouge_score import rouge_scorer

# Comparar respuestas generadas vs. esperadas
reference = "El cambio climático está afectando las temperaturas globales."
generated = rag_query("¿Cómo afecta el cambio climático al planeta?")

scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
scores = scorer.score(reference, generated)

print("\nMétricas de Evaluación de Generación:")
print(scores)


ROUGE mide la similitud entre la respuesta generada y la respuesta esperada.
ROUGE alto indica que el modelo genera respuestas precisas.

# Conclusiones

* Creamos un sistema RAG real con ChromaDB y OpenAI para búsqueda de noticias.
* Probamos consultas reales y generamos respuestas con GPT-4.
* Evaluamos la recuperación con Recall@K y la generación con ROUGE.

RETO: (1) actualización automática de noticias - (2) visualización de embeddings